Fake News Classifier

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv', engine='python', error_bad_lines=False)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
Skipping line 6436: unexpected end of data


In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


# Handle with null data

In [4]:
df.isnull().sum()

id          0
title     178
author    619
text       13
label       0
dtype: int64

In [5]:
df.shape

(6434, 5)

In [9]:
df=df.dropna()

In [10]:
df.shape

(5637, 5)

In [11]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

# Splitting dependent and independent features

In [12]:
x=df.drop('label',axis=1)

In [13]:
x.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [14]:
x.shape

(5637, 4)

In [15]:
y=df['label']

In [16]:
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [17]:
import tensorflow as tf

In [19]:
tf.__version__

'2.9.2'

In [21]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import LSTM
from tensorflow.keras.preprocessing.text import one_hot

In [22]:
# Vacobulary size
voc_size=5000

In [30]:
messages=x.copy()

In [47]:
messages.isnull().sum()

id        0
title     0
author    0
text      0
dtype: int64

In [33]:
import nltk
import re
from nltk.corpus import stopwords

In [50]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [52]:
messages.reset_index(inplace=True)

In [53]:
# Data Preprocessing
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0,len(messages)):
  review=re.sub('[^a-zA-Z]',' ',messages['title'][i])
  review=review.lower()
  review=review.split()

  review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)

In [54]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [55]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]

In [57]:
onehot_repr

[[2362, 493, 490, 1171, 167, 4325, 552, 1441, 313, 2633],
 [4651, 126, 2465, 1130, 462, 997, 4024],
 [3129, 3892, 1961, 3003],
 [3709, 1984, 180, 2280, 3958, 3334],
 [887, 462, 4977, 1007, 2125, 1325, 462, 1426, 4262, 1429],
 [4397,
  1575,
  4949,
  271,
  2354,
  4299,
  309,
  645,
  3772,
  4805,
  2159,
  2725,
  2425,
  4582,
  4024],
 [192, 1318, 2091, 1967, 610, 1802, 4526, 1170, 3025, 2001, 2341],
 [2843, 597, 2991, 1686, 4573, 3295, 4299, 2621, 3025, 2001, 2341],
 [121, 1140, 441, 4421, 341, 4477, 4230, 3724, 4299, 141],
 [2994, 1094, 2169, 4077, 248, 4691, 208, 3409],
 [3327, 1736, 1723, 2099, 3869, 2341, 2060, 3917, 454, 1847, 3900],
 [2280, 3077, 167, 4477, 4299, 4573],
 [1500, 2726, 1668, 1361, 2596, 193, 4651, 2849, 2959],
 [1822, 3277, 2166, 2901, 2359, 1174, 1516, 3025, 2001, 2341],
 [274, 1637, 1660, 2133, 4031, 3025, 2001, 2341],
 [292, 1615, 4284, 2740, 4918, 3398, 1660, 2273, 2794, 1230],
 [3840, 4907, 126],
 [167, 2406, 346, 2684, 4299, 4352, 2011, 4024],
 [3433, 

# Embedding Representation

In [58]:
sent_length=20

In [60]:
emb_doc=pad_sequences(onehot_repr, padding='pre',maxlen=sent_length)

In [61]:
emb_doc

array([[   0,    0,    0, ..., 1441,  313, 2633],
       [   0,    0,    0, ...,  462,  997, 4024],
       [   0,    0,    0, ..., 3892, 1961, 3003],
       ...,
       [   0,    0,    0, ..., 2363, 2694,  226],
       [   0,    0,    0, ..., 1526, 4348, 3626],
       [   0,    0,    0, ..., 4299,  463,  126]], dtype=int32)

In [67]:
# Creating model
model=Sequential()
model.add(Embedding(voc_size,40,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 200)              112800    
 nal)                                                            
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [68]:
import numpy as np
x_final=np.array(emb_doc)
y_final=np.array(y)

In [69]:
# Train test split
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_final,y_final,test_size=0.33,random_state=42)

In [71]:
# Model training
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
118/118 [==============================] - 10s 16ms/step - loss: 0.3934 - accuracy: 0.7985 - val_loss: 0.2276 - val_accuracy: 0.9135
Epoch 2/10
118/118 [==============================] - 1s 9ms/step - loss: 0.1421 - accuracy: 0.9468 - val_loss: 0.2355 - val_accuracy: 0.8936
Epoch 3/10
118/118 [==============================] - 1s 9ms/step - loss: 0.0537 - accuracy: 0.9833 - val_loss: 0.2960 - val_accuracy: 0.8974
Epoch 4/10
118/118 [==============================] - 1s 9ms/step - loss: 0.0173 - accuracy: 0.9950 - val_loss: 0.4755 - val_accuracy: 0.8963
Epoch 5/10
118/118 [==============================] - 1s 9ms/step - loss: 0.0181 - accuracy: 0.9934 - val_loss: 0.4439 - val_accuracy: 0.8839
Epoch 6/10
118/118 [==============================] - 1s 9ms/step - loss: 0.0078 - accuracy: 0.9976 - val_loss: 0.6035 - val_accuracy: 0.8721
Epoch 7/10
118/118 [==============================] - 1s 9ms/step - loss: 0.0080 - accuracy: 0.9976 - val_loss: 0.6428 - val_accuracy: 0.8877
Epoc

In [72]:
y_predict=model.predict(x_test)

59/59 [==============================] - 1s 3ms/step


In [73]:
y_predict

array([[5.2244544e-05],
       [8.2175638e-06],
       [7.1844850e-05],
       ...,
       [1.7512347e-04],
       [7.4543859e-05],
       [4.5726300e-04]], dtype=float32)

In [74]:
import numpy as np
y_predict=np.where(y_predict>=0.5,1,0)

In [75]:
y_predict

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [79]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_predict)

array([[952, 105],
       [107, 697]])

In [80]:
accuracy_score(y_test,y_predict)

0.8860827512090274